# Face and Emotion Recognition Live Camera

## Imports

In [ ]:
import os
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


## Data Preprocessing

In [ ]:
parent_directory = "D:\\Deep learning project\\our data"
os.listdir(parent_directory)

# Get a list of all directories within the parent directory
directories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

# Get the full paths of each directory
directory_paths = [os.path.join(parent_directory, d) for d in directories]

# Print the paths of directories
# print("Paths of directories within", parent_directory, "are:")
print(directory_paths)



['D:\\Deep learning project\\our data\\Peter', 'D:\\Deep learning project\\our data\\Sandra']


In [ ]:
# Specify the directory containing your images
# directory_path = 'path/to/directory/with/images'

# Create an empty list to store image arrays
image_arrays = []
image_names = []
folder_names = []
# Iterate through images in the directory
for path in directory_paths:
    for filename in os.listdir(path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Consider only image files
            image_path = os.path.join(path, filename)

            # Read the image and convert it to a NumPy array
            image = cv2.imread(image_path)

            if image is not None:
                resized_image = cv2.resize(image, (100, 100))
                image_arrays.append(resized_image)

                # Extract the image name without extension
                name_without_extension = os.path.splitext(filename)[0]
                name_without_extension = ''.join(filter(str.isalpha, name_without_extension))
                image_names.append(name_without_extension)

                folder_name = os.path.basename(os.path.dirname(image_path))
                folder_names.append(folder_name)
            else:
                print(f"Failed to read {filename}")

# Now image_arrays contains NumPy arrays of each image in the directory
# Access them like image_arrays[0], image_arrays[1], etc.


NameError: name 'directory_paths' is not defined

In [ ]:
import os
import cv2
import numpy as np

# Specify the parent directory containing your image directories
parent_directory = '/content/drive/MyDrive/marinaface'

# Get a list of all directories within the parent directory"paths of photos"
directories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

# Create empty lists to store image arrays, image names, and folder names
image_arrays = []
folder_names = []

# Iterate through directories
for directory in directories:
    # Get the full path of the directory
    directory_path = os.path.join(parent_directory, directory)

    # Iterate through images in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Consider only image files
            image_path = os.path.join(directory_path, filename)

            # Read the image and convert it to a NumPy array
            image = cv2.imread(image_path)

            if image is not None:
                resized_image = cv2.resize(image, (100, 100))
                image_arrays.append(resized_image)

                # Extract the image name without extension
                name_without_extension = os.path.splitext(filename)[0]
                name_without_extension = ''.join(filter(str.isalpha, name_without_extension))
                image_names.append(name_without_extension)

                folder_names.append(directory)
            else:
                print(f"Failed to read {filename}")

# Convert lists to NumPy arrays
image_arrays = np.array(image_arrays)
image_names = np.array(image_names)
folder_names = np.array(folder_names)

# Now image_arrays contains NumPy arrays of each image in the directories
# Access them like image_arrays[0], image_arrays[1], etc.


In [ ]:
print("Parent directory:", parent_directory)
print("List of directories:", directories)


Parent directory: /content/drive/MyDrive/marinaface
List of directories: ['marina']


In [ ]:

print("Files in 'marinaface' folder:")
for file in files:
    print(file)

Files in 'marinaface' folder:
WhatsApp Image 2024-02-28 at 9.36.08 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.13 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.13 PM (1).jpeg
WhatsApp Image 2024-02-28 at 9.36.13 PM (2).jpeg
WhatsApp Image 2024-02-28 at 9.36.14 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.14 PM (1).jpeg
WhatsApp Image 2024-02-28 at 9.36.14 PM (2).jpeg
WhatsApp Image 2024-02-28 at 9.36.14 PM (3).jpeg
WhatsApp Image 2024-02-28 at 9.36.16 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.17 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.17 PM (1).jpeg
WhatsApp Image 2024-02-28 at 9.36.18 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.18 PM (1).jpeg
WhatsApp Image 2024-02-28 at 9.36.19 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.19 PM (1).jpeg
WhatsApp Image 2024-02-28 at 9.36.20 PM.jpeg
WhatsApp Image 2024-02-28 at 9.36.21 PM.jpeg


In [ ]:
print(directories)

['marina']


In [ ]:
print(image_arrays)

[]


## Data Augmentation

In [ ]:
augmented_images = []  # Array to store augmented images
augmented_person_labels = []  # Array to store person labels for augmented images
augmented_emotion_labels = []  # Array to store emotion labels for augmented images

# Custom augmentation functions
def flip_horizontally(image):
    return cv2.flip(image, 1)  # Flip horizontally

def rotate(image, angle=30):
    rows, cols, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(image, rotation_matrix, (cols, rows))

def change_brightness(image, brightness_factor=0.5):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * brightness_factor
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def add_noise(image):
    noise = np.random.normal(0, 20, image.shape)
    noisy_image = np.clip(image + noise, 0, 255)
    return noisy_image.astype(np.uint8)

# List of augmentation functions
augmentation_functions = [flip_horizontally, rotate, change_brightness, add_noise]



In [ ]:
num = 10

for idx, img in enumerate(image_arrays):
    emotion_label = image_names[idx]
    person_label = folder_names[idx]

    # Apply multiple augmentations to each image
    for  _ in range(num):
        for aug_func in augmentation_functions:
            augmented_img = aug_func(img)

            # Store augmented image and labels
            augmented_images.append(augmented_img)
            augmented_person_labels.append(person_label)
            augmented_emotion_labels.append(emotion_label)

In [ ]:
# augmented_person_labels[239]

In [ ]:
# augmented_emotion_labels[239]

In [ ]:
# cv2.imshow('img',augmented_images[239])
# cv2.waitKey(0)

In [ ]:
# len(augmented_images)

In [ ]:

image_arrays = np.array(image_arrays + augmented_images)
image_names =  np.array(image_names + augmented_emotion_labels)
folder_names =  np.array(folder_names + augmented_person_labels )

## Normalization

In [ ]:
image_arrays = image_arrays / 255

## Our Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, BatchNormalization, Dropout

# Define input shape for the images
input_shape = (100, 100, 3)  # Assuming RGB images of size 224x224

# Shared input layer
input_layer = Input(shape=input_shape)

# Shared convolutional layers for feature extraction
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
drp1 = Dropout(0.25)(conv1)
pool1 = MaxPooling2D((2, 2))(drp1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
drp2 = Dropout(0.25)(conv2)
pool2 = MaxPooling2D((2, 2))(drp2)


# Branch 1: Face Recognition
flatten = Flatten()(pool2)
dense1 = Dense(128, activation='relu')(flatten)
drp2 = Dropout(0.25)(dense1)
dense2 = Dense(128, activation='relu')(dense1)

output_face_recognition = Dense(2, activation='softmax', name='face_recognition')(dense2)  # num_classes_face is the number of face identities

# Branch 2: Emotion Recognition
conv2 = Conv2D(64, (3, 3), activation='relu')(pool2)
norm1 = BatchNormalization()(conv2)
drp2 = Dropout(0.25)(norm1)
pool2 = MaxPooling2D((2, 2))(drp2)
drp2 = Dropout(0.25)(pool2)
pool2 = MaxPooling2D((2, 2))(drp2)

flatten = Flatten()(pool2)
dense3 = Dense(128, activation='relu')(flatten)
drp2 = Dropout(0.25)(dense3)
dense4 = Dense(128, activation='relu')(drp2)
drp2 = Dropout(0.25)(dense4)
dense5 = Dense(128, activation='relu')(drp2)
output_emotion_recognition = Dense(3, activation='softmax', name='emotion_recognition')(dense5)  # num_classes_emotion is the number of emotion classes

# Combine both branches
combined_model = tf.keras.Model(inputs=input_layer, outputs=[output_face_recognition, output_emotion_recognition])

# Compile the model with appropriate loss functions and metrics for each branch
combined_model.compile(optimizer='adam',
                       loss={'face_recognition': 'categorical_crossentropy', 'emotion_recognition': 'categorical_crossentropy'},
                       metrics={'face_recognition': 'accuracy', 'emotion_recognition': 'accuracy'})


## Label Encoder

In [ ]:
label_encoder = LabelEncoder()
face_label_encoded = label_encoder.fit_transform(folder_names)
y_face_one_hot = to_categorical(face_label_encoded, num_classes=2)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# emotions_label contains string labels like 'Happy', 'Sad', 'Neutral'
# Convert string labels to numerical labels using LabelEncoder
label_encoder = LabelEncoder()
emotions_label_encoded = label_encoder.fit_transform(image_names)

# Now, emotions_label_encoded contains numerical labels (0, 1, 2, etc.) corresponding to different emotions
# Use to_categorical to convert these numerical labels to one-hot encoded format
# num_classes_emotion = len(np.unique(emotions_label_encoded))  # Calculate the number of unique classes
y_emotion_one_hot = to_categorical(emotions_label_encoded, num_classes=3)
y_emotion_one_hot

array([], shape=(0, 3), dtype=float32)

In [ ]:
print("Size of image_arrays:", len(image_arrays))


Size of image_arrays: 0


In [ ]:
x_face_train, x_face_test, y_face_train, y_face_test = train_test_split(image_arrays, y_face_one_hot, test_size=0.1, random_state=42)
#x_emotion_train, x_emotion_test, y_emotion_train, y_emotion_test = train_test_split(image_arrays, y_emotion_one_hot, test_size=0.1, random_state=42)

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
y_face_train.shape, y_emotion_train.shape, x_face_train.shape

((4317, 2), (4317, 3), (4317, 100, 100, 3))

## Train the model

In [ ]:
combined_model.fit(x_face_train, {'face_recognition': y_face_train, 'emotion_recognition': y_emotion_train},
                   epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
108/108 [==============================] - 45s 404ms/step - loss: 0.9952 - face_recognition_loss: 0.0965 - emotion_recognition_loss: 0.8987 - face_recognition_accuracy: 0.9684 - emotion_recognition_accuracy: 0.5694 - val_loss: 1.0398 - val_face_recognition_loss: 0.0050 - val_emotion_recognition_loss: 1.0349 - val_face_recognition_accuracy: 1.0000 - val_emotion_recognition_accuracy: 0.5382
Epoch 2/10
108/108 [==============================] - 43s 399ms/step - loss: 0.3940 - face_recognition_loss: 0.0316 - emotion_recognition_loss: 0.3623 - face_recognition_accuracy: 0.9936 - emotion_recognition_accuracy: 0.8477 - val_loss: 0.7484 - val_face_recognition_loss: 0.0012 - val_emotion_recognition_loss: 0.7472 - val_face_recognition_accuracy: 1.0000 - val_emotion_recognition_accuracy: 0.7477
Epoch 3/10
108/108 [==============================] - 42s 392ms/step - loss: 0.1710 - face_recognition_loss: 3.9440e-04 - emotion_recognition_loss: 0.1706 - face_recognition_accuracy: 1.0000 - e

In [ ]:
face_evaluation = combined_model.evaluate(x_face_test, (y_face_test, y_emotion_test))

15/15 [==============================] - 1s 40ms/step - loss: 0.1892 - face_recognition_loss: 1.3923e-05 - emotion_recognition_loss: 0.1892 - face_recognition_accuracy: 1.0000 - emotion_recognition_accuracy: 0.9354


In [ ]:
# index = 24
# face_arr = ["Peter", "Sandra"]
# emotion_arr = ["Happy", "Neutral", "Sad"]
# predictions = combined_model.predict(x_face_test)
# face_predictions, emotion_predictions = predictions
# face_prediction_for_image = face_predictions[index]
# emotion_prediction_for_image = emotion_predictions[index]

# # Use argmax to get the predicted classes (assuming it's classification)
# predicted_face_class = np.argmax(face_prediction_for_image)
# predicted_emotion_class = np.argmax(emotion_prediction_for_image)


In [ ]:
# face_arr[predicted_face_class], emotion_arr[predicted_emotion_class]

In [ ]:
# cv2.imshow('img',x_face_test[24])
# cv2.waitKey(0)

In [ ]:
# emotion_predictions[1]

array([9.9999994e-01, 5.4493441e-08, 4.0653614e-10], dtype=float32)

In [ ]:
# x_face_test.shape

(480, 100, 100, 3)

In [ ]:
# test_im_path = "D:\\Deep learning project\\p_happy21.jpg"
# img = cv2.imread(test_im_path)
# res_img = cv2.resize(img, (100, 100))
# res_img = res_img.reshape((1,) + res_img.shape)
# res_img.shape

In [ ]:
# cv2.imshow('im', res_img)
# cv2.waitKey(0)

In [ ]:
# predictions = combined_model.predict(res_img)
# face_predictions, emotion_predictions = predictions
# face_prediction_for_image = face_predictions
# emotion_prediction_for_image = emotion_predictions
# predicted_face_class = np.argmax(face_prediction_for_image)
# predicted_emotion_class = np.argmax(emotion_prediction_for_image)

In [ ]:
# face_predictions, emotion_predictions

In [ ]:
# predictions

In [ ]:
face_arr[predicted_face_class], emotion_arr[predicted_emotion_class]


('Peter', 'Neutral')

In [ ]:
predictions

[array([[1., 0.]], dtype=float32), array([[0., 1., 0.]], dtype=float32)]

## Live camera

In [ ]:
import cv2
import numpy as np

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the video capture
cap = cv2.VideoCapture(0)  # 0 for default camera, you can change it to the appropriate camera index if you have multiple cameras

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Ensure the frame was read properly
    if not ret:
        print("Failed to capture frame")
        break

    # Convert the frame to grayscale for face detection
    # gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=10, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face region
        face_roi = frame[y:y + h, x:x + w]

        # Preprocess the face (resize it to the input shape expected by your model)
        resized_face = cv2.resize(face_roi, (100, 100))

        # Normalize the face
        normalized_face = resized_face / 255.0

        # Reshape the face to match the model's input shape
        input_face = np.reshape(normalized_face, (1, 100, 100, 3))

        # Make predictions using the combined model
        predictions = combined_model.predict(input_face)
        face_predictions, emotion_predictions = predictions

        # Get the predicted classes
        predicted_face_class = np.argmax(face_predictions)
        predicted_emotion_class = np.argmax(emotion_predictions)

        # Display the predicted classes
        cv2.putText(frame, f"Person: {face_arr[predicted_face_class]}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Emotion: {emotion_arr[predicted_emotion_class]}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame with predictions
    cv2.imshow('Video Capture', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step


## Model Prediction

In [ ]:
predictions = combined_model.predict(x_face_test)
face_predictions, emotion_predictions = predictions


predicted_face_class = np.argmax(face_predictions, axis=1)
predicted_emotion_class = np.argmax(emotion_predictions, axis=1)

15/15 [==============================] - 1s 40ms/step


In [ ]:
face_predictions

array([[9.99999583e-01, 3.18943052e-07],
       [9.99376118e-01, 6.23864704e-04],
       [9.99999940e-01, 8.14837864e-10],
       [9.99999821e-01, 1.14501091e-07],
       [9.99999702e-01, 2.80175698e-07],
       [9.99996245e-01, 3.67064422e-06],
       [9.99999344e-01, 5.97833321e-07],
       [9.99999940e-01, 3.70471362e-12],
       [9.99827862e-01, 1.72251835e-04],
       [9.99999940e-01, 8.14898482e-10],
       [9.99999940e-01, 6.00586869e-09],
       [9.99999940e-01, 5.59024578e-08],
       [9.99999940e-01, 1.40847153e-10],
       [9.99964297e-01, 3.56333039e-05],
       [9.99998748e-01, 1.16777960e-06],
       [9.99999940e-01, 3.52705376e-09],
       [9.99993980e-01, 5.95772963e-06],
       [9.99999344e-01, 5.96132736e-07],
       [9.99992311e-01, 7.62817490e-06],
       [9.99957621e-01, 4.22637713e-05],
       [9.99999940e-01, 2.23534768e-09],
       [9.99997556e-01, 2.43824275e-06],
       [9.99998987e-01, 9.23759956e-07],
       [1.85415065e-05, 9.99981344e-01],
       [9.999998

## Handle Model Prediction

In [ ]:
zeros = np.zeros_like(face_predictions)
zeros[ np.arange(len(face_predictions)), predicted_face_class] = 1
predicted_faces = zeros

In [ ]:
zeros = np.zeros_like(emotion_predictions)
zeros[ np.arange(len(emotion_predictions)), predicted_emotion_class] = 1
predicted_emotions = zeros
predicted_emotions

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

## Precision, recall and F1-score

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


# Calculate precision, recall, and F1-score for face recognition
face_precision = precision_score(y_face_test, predicted_faces, average='micro')
face_recall = recall_score(y_face_test, predicted_faces, average='macro')
face_f1 = f1_score(y_face_test, predicted_faces, average='weighted')

print("Face Recognition Metrics:")
print(f"Precision: {face_precision:.4f}, Recall: {face_recall:.4f}, F1-score: {face_f1:.4f}")


Face Recognition Metrics:
Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000


In [ ]:
# Calculate precision, recall, and F1-score for face recognition
emotion_precision = precision_score(y_emotion_test, predicted_emotions, average='micro')
emotion_recall = recall_score(y_emotion_test, predicted_emotions, average='macro')
emotion_f1 = f1_score(y_emotion_test, predicted_emotions, average='weighted')

print("emotion Recognition Metrics:")
print(f"Precision: {emotion_precision:.4f}, Recall: {emotion_recall:.4f}, F1-score: {emotion_f1:.4f}")

emotion Recognition Metrics:
Precision: 0.9354, Recall: 0.9193, F1-score: 0.9329


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('Perfect_one.h5')

15/15 [==============================] - 1s 34ms/step


In [ ]:
# import numpy as np

# # Sample prediction probabilities (replace this with your actual predicted probabilities)
# predicted_probabilities = np.array([[0.3, 0.7], [0.6, 0.4], [0.2, 0.8]])

# # Get the index of the maximum probability along axis 1 (assuming binary classification)
# max_prob_indices = np.argmax(predicted_probabilities, axis=1)

# # Create a one-zero array where the position of the largest probability is marked as one
# one_zero_array = np.zeros_like(predicted_probabilities)
# one_zero_array[np.arange(len(predicted_probabilities)), max_prob_indices] = 1

# print("Original Probabilities:")
# print(predicted_probabilities)
# print("\nOne-Zero Array:")
# print(one_zero_array)
